In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import requests
import re
import sqlite3
options = webdriver.ChromeOptions()
#options.add_argument('headless')
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('lang=ko_KR')
options.add_argument('authority= www.coupang.com')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36')
options.add_argument('accept=text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9')
options.add_argument('cookie=sid=e75eeb9defde46139683c18adb35203a948c0d3e; PCID=27432903235840507024460; overrideAbTestGroup=%5B%5D; MARKETID=27432903235840507024460; bm_sz=DC5C34CE31904F0FF94A2D8E148848B0~YAAQddojF7tk13N4AQAAg2jweAvPajrDO/uPUYwdXozHv2SFFe9iyIMiMNOChTb9CQtH9FmRzI0N/noAJXr+UaQKp0qAou3Ru00JPX4Iat/lXTO8+sNlheJOlAdNoxObgqjtKtZXgiaG4/v9x0OmBLP5bBZueV3kts5o3adaFDgcL7Xh8QEelY2VAlTUAXrZ1Q==; _fbp=fb.1.1616936724605.320452491; bm_sv=B7B343F763238B2AB780F1C1727D5625~CBc8xXUQu1Ukozdr08cb8NIeyDJRoP1cLpv4IBRMxM5n3qRbKiJnEGWtCPoF6bz9kc9so+5W7QZOPly+rEvgYlZrb/TIxA85Vkx+5tiTD871m2T0693WmnKhU+TznY/S/iaL2/+evFuzyktw6c7Fx2ebx8m3Fs3SYof28f0hgMw=; ak_bmsc=731321ED51499E0E7F7BE5F947876BC01723DA75C44F0000147F6060A836AC6C~plRhbR98IP8fD8TSTwa1N47Lg32D/corCNS/et/K0XrpVFgLnej4X0+mg/4XdqbyoDPC8/dyENsaY6+CsIM0dhCi+fwXn4rSPxp7MCxi3pvzSR2Cz8SCLPI5igO1qQAF2eW10YtcZjegz/19wFa4EatSJEdSi0Rt2OwdIDetOqDFgC7JTZ0Pt6dGVtNCqPKUFgmNl+/DV1O2+VOYZUVWTt8E54Li0XjLrQl+bPoSexLY1sUsRHg8b8ZdoteBA72ZTV+um1nYfxDhIjTlt9gi+eJ14Ofmm8j6Wh83JkQ4nKVNk=; _abck=447EC68AFE4E8CA6A5C6A0B3E3F67B09~0~YAAQddojF+1l13N4AQAAHG3weAU1LQrI4XvEe5Bf24dUMBU/1pcNI931S9CoSmJQK6Rznvc6fry2bmcWl0Q9NIy3NhaxfGsWlRPB6Qu5qo1OAauX+oYFu0+91XAtiaQuE3Le3SnLDXBgp1ZYLbqPAhbA6mOkFDOcbRcHKvb87j1OQLsM6/B7xFC20LdCviWnwHaZ1850G0EeQak+/IwVISPF0x9pyh7uu/5lMpu3be3n2P6jP91dOCH4AAwOFad0WqTvAPYnGDz5A55OYc2IN/cOQJqg0Xx6dDoVeLiuwWUxikoAOJ1luJiCp93YX9r2LEOxSsuZAkBfI6kTzplfRK1r72DXd2JUUYUb4tWB6qbkED3zqOfNXhRfFoyOj7gTp0YJ2m3BQk+n53mLTahGDK56Ulj8rp+83Q==~-1~-1~-1; searchSoterTooltip=OFF; baby-isWide=wide')

In [2]:
def tototo(review3):
    if len(review3)==2:
        if review3[0].text=='아주 만족해요':
            reviewF=5
        elif review3[0].text=='적당히 만족해요':
            reviewF=3
        else:
            reviewF=1
        if review3[1].text=='예상보다 오래 남아요':
            reviewT=5
        elif review3[1].text=='적당히 지속 돼요':
            reviewT=3
        else:
            reviewT=1
    elif len(review3)==1:
        if review3[0].text=='아주 만족해요':
            reviewF=5
            reviewT=''
        elif review3[0].text=='적당히 만족해요':
            reviewF=3
            reviewT=''
        elif review3[0].text=='만족스럽지 않아요':
            reviewF=1
            reviewT=''
        elif review3[0].text=='예상보다 오래 남아요':
            reviewT=5
            reviewF=''
        elif review3[0].text=='적당히 지속 돼요':
            reviewT=3
            reviewF=''
        else:
            reviewT=1
            reviewF=''
    else:
        reviewT=''
        reviewF=''
    return reviewT,reviewF

In [ ]:
def insMain(d):
    with sqlite3.connect('coupang2.db') as conn:
        cursor=conn.cursor()
        sql='insert into coupang2 (name,brand_name,review,rate,Frag,Frag_time) values(?,?,?,?,?,?)'
        res=cursor.execute(sql,d)
        print('.',end='')

In [ ]:
def lololo(review):
    if review is None:
        review=' '
    else:
        if review.text is None:
            review=' '
        else:
            review=review.text.replace('\n','').replace('\t',"").replace('\r',"").replace('  ',"").strip().strip(' ')
    return review

In [ ]:
def getbrand(html2):
    try:
        brand=html2.find(attrs={'class':'prod-brand-name'})['data-brand-name']
        return brand
    except AttributeError as e:
        return ''

In [ ]:
###url 가져오기
###남자 향수
for i in range(1,18):
    url = 'https://www.coupang.com/np/categories/176600?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page={}&channel=user&fromComponent=Y&selectedPlpKeepFilter=&sorter=bestAsc&filter=&component=176500&rating=0'.format(i)
    headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36"}
    res=requests.get(url,headers=headers)
    src=res.content.strip()
    html =BeautifulSoup(src,'html.parser')
    listt = html.find(attrs={'class':'baby-product-list'})
    lit = listt.find_all('li')
    for lt in lit:
        a=lt.find(attrs={'class':'baby-product-link'})['href']
        #rate=lt.find(attrs={'class':'rating'})
        url2='https://www.coupang.com'+lt.find(attrs={'class':'baby-product-link'})['href']+'&isAddedCart='
        res2=requests.get(url2,headers=headers)
        src2=res2.content.strip()
        html2 =BeautifulSoup(src2,'html.parser')
        brand = getbrand(html2)
        product=html2.find(attrs={'class':'prod-buy-header__title'}).text
        product=re.sub('\[.*?\]','',product)
        product=re.sub('\<.*?\>','',product)
        product=re.sub(',.*?','',product)
        product=re.sub('\+.*?','',product)
        aa=a[13:25]
        result = re.sub('[^0-9]', '', aa)
        t=0
        for i in range(1,2000):
            url = 'https://www.coupang.com/vp/product/reviews?productId={0}&page={1}&size=5&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=3&ratingSummary=true'.format(result,i)
            headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36"}
            res=requests.get(url,headers=headers)
            src=res.content.strip()
            html =BeautifulSoup(src,'html.parser')
            tgts=html.find_all(attrs={'class':'sdp-review__article__list js_reviewArticleReviewList'})
            print(t)
            t+=1
            if len(tgts)>0:
                for tgt in tgts:
                    review1=tgt.find(attrs={'class':'sdp-review__article__list__headline'})
                    rate=tgt.find(attrs={'class':'sdp-review__article__list__info__product-info__star-orange js_reviewArticleRatingValue'})
                    rate=rate['data-rating']
                    review1=lololo(review1)
                    review2=tgt.find(attrs={'class':'sdp-review__article__list__review__content js_reviewArticleContent'})
                    review2=lololo(review2)
                    review3=tgt.find_all(attrs={'class':'sdp-review__article__list__survey__row__answer'})
                    reviewT,reviewF=tototo(review3)
                    review_all=str(review1)+str(review2)
                    #print('평점 : '+rate)
                    if len(brand)==0:
                        brand=product.split(' ')[0]
                    #print('브랜드 : '+brand)
                    #print('상품명 : '+product)
                    #print(review1.text)#review2)
                    #print(review2.text)
                    #print(review3[0].text.replace('\n','').strip()+review3[1].text.replace('\n','').strip())
                    #name,brand_name,review,rate,Frag,Frag_time
                    d=(product,brand,review_all,rate,reviewF,reviewT)
                    insMain(d)
                    print('##'*50)
            else:
                break

In [ ]:
###url 가져오기
###여자 향수
for i in range(1,18):
    url = 'https://www.coupang.com/np/categories/176599?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page={}&channel=user&fromComponent=Y&selectedPlpKeepFilter=&sorter=bestAsc&filter=&component=176499&rating=0'.format(i)
    headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36"}
    res=requests.get(url,headers=headers)
    src=res.content.strip()
    html =BeautifulSoup(src,'html.parser')
    listt = html.find(attrs={'class':'baby-product-list'})
    lit = listt.find_all('li')
    for lt in lit:
        a=lt.find(attrs={'class':'baby-product-link'})['href']
        #rate=lt.find(attrs={'class':'rating'})
        url2='https://www.coupang.com'+lt.find(attrs={'class':'baby-product-link'})['href']+'&isAddedCart='
        res2=requests.get(url2,headers=headers)
        src2=res2.content.strip()
        html2 =BeautifulSoup(src2,'html.parser')
        brand=html2.find(attrs={'class':'prod-brand-name'})['data-brand-name']
        product=html2.find(attrs={'class':'prod-buy-header__title'}).text
        product=re.sub('\[.*?\]','',product)
        product=re.sub('\<.*?\>','',product)
        product=re.sub('\+.*?','',product)
        product=re.sub(',.*?','',product)
        product=product.strip()
        aa=a[13:25]
        result = re.sub('[^0-9]', '', aa)
        t=0
        for i in range(1,2000):
            url = 'https://www.coupang.com/vp/product/reviews?productId={0}&page={1}&size=5&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=3&ratingSummary=true'.format(result,i)
            headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36"}
            res=requests.get(url,headers=headers)
            src=res.content.strip()
            html =BeautifulSoup(src,'html.parser')
            tgts=html.find_all(attrs={'class':'sdp-review__article__list js_reviewArticleReviewList'})
            print(t)
            t+=1
            if len(tgts)>0:
                for tgt in tgts:
                    review1=tgt.find(attrs={'class':'sdp-review__article__list__headline'})
                    rate=tgt.find(attrs={'class':'sdp-review__article__list__info__product-info__star-orange js_reviewArticleRatingValue'})
                    rate=rate['data-rating']
                    review1=lololo(review1)
                    review2=tgt.find(attrs={'class':'sdp-review__article__list__review__content js_reviewArticleContent'})
                    review2=lololo(review2)
                    review3=tgt.find_all(attrs={'class':'sdp-review__article__list__survey__row__answer'})
                    reviewT,reviewF=tototo(review3)
                    review_all=str(review1)+str(review2)
                    #print('평점 : '+rate)
                    if len(brand)==0:
                        brand=product.split(' ')[0]
                    #print('브랜드 : '+brand)
                    #print('상품명 : '+product)
                    #print(review1.text)#review2)
                    #print(review2.text)
                    #print(review3[0].text.replace('\n','').strip()+review3[1].text.replace('\n','').strip())
                    #name,brand_name,review,rate,Frag,Frag_time
                    d=(product,brand,review_all,rate,reviewF,reviewT)
                    insMain(d)
            else:
                break

## 감

In [ ]:
###url 가져오기
###남자 향수
### 예시
url = 'https://www.coupang.com/np/categories/176600?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page={}&channel=user&fromComponent=Y&selectedPlpKeepFilter=&sorter=bestAsc&filter=&component=176500&rating=0'.format(i)
headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36"}
res=requests.get(url,headers=headers)
src=res.content.strip()
html =BeautifulSoup(src,'html.parser')
listt = html.find(attrs={'class':'baby-product-list'})
lit = listt.find_all('li')
for lt in lit:
    a=lt.find(attrs={'class':'baby-product-link'})['href']
    #rate=lt.find(attrs={'class':'rating'})
    url2='https://www.coupang.com'+lt.find(attrs={'class':'baby-product-link'})['href']+'&isAddedCart='
    res2=requests.get(url2,headers=headers)
    src2=res2.content.strip()
    html2 =BeautifulSoup(src2,'html.parser')
    brand=html2.find(attrs={'class':'prod-brand-name'})['data-brand-name']
    product=html2.find(attrs={'class':'prod-buy-header__title'}).text
    product=re.sub('\[.*?\]','',product)
    product=re.sub(',.*?','',product)
    aa=a[13:25]
    result = re.sub('[^0-9]', '', aa)
    t=0
    for i in range(1,2000):
        url = 'https://www.coupang.com/vp/product/reviews?productId={0}&page={1}&size=5&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=3&ratingSummary=true'.format(result,i)
        headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36"}
        res=requests.get(url,headers=headers)
        src=res.content.strip()
        html =BeautifulSoup(src,'html.parser')
        tgts=html.find_all(attrs={'class':'sdp-review__article__list js_reviewArticleReviewList'})
        print(t)
        t+=1
        if len(tgts)>0:
            for tgt in tgts:
                review1=tgt.find(attrs={'class':'sdp-review__article__list__headline'})
                rate=tgt.find(attrs={'class':'sdp-review__article__list__info__product-info__star-orange js_reviewArticleRatingValue'})
                rate=rate['data-rating']
                review1=lololo(review1)
                review2=tgt.find(attrs={'class':'sdp-review__article__list__review__content js_reviewArticleContent'})
                review2=lololo(review2)
                review3=tgt.find_all(attrs={'class':'sdp-review__article__list__survey__row__answer'})
                reviewT,reviewF=tototo(review3)
                review_all=str(review1)+str(review2)
                #print('평점 : '+rate)
                if len(brand)==0:
                    brand=product.split(' ')[0]
                #print('브랜드 : '+brand)
                #print('상품명 : '+product)
                #print(review1.text)#review2)
                #print(review2.text)
                #print(review3[0].text.replace('\n','').strip()+review3[1].text.replace('\n','').strip())
                #name,brand_name,review,rate,Frag,Frag_time
                d=(product,brand,review_all,rate,reviewF,reviewT)
                insMain(d)
                print('##'*50)
        else:
            break